# Rent by SA2

## Import Library

In [8]:
import  pandas as pd
import geopandas as gpd
import folium
import json

## Read data

In [9]:
sa2_rent_data = pd.read_csv('../../data/raw/domain_data/merged_final_geo.csv')

In [10]:
shp_sa2 = gpd.read_file('../../data/landing/region_data/sa2_dataset/sa2_unzip/SA2_2021_AUST_GDA2020.shp')
victoria_sa2 = shp_sa2[shp_sa2['STE_NAME21'] == 'Victoria']

In [11]:
victoria_sa2

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
644,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5..."
645,201011002,Ballarat,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,12.3787,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.81896 -37.55582, 143.81644 -37.5..."
646,201011005,Buninyong,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,51.5855,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.84171 -37.61596, 143.84176 -37.6..."
647,201011006,Delacombe,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,34.1607,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.75050 -37.59119, 143.75044 -37.5..."
648,201011007,Smythes Creek,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,104.7274,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.73296 -37.62333, 143.73263 -37.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,217041478,Moyne - West,0,No change,21704,Warrnambool,217,Warrnambool and South West,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,2037.7626,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((142.00870 -38.41715, 142.00876..."
1164,217041479,Warrnambool - North,0,No change,21704,Warrnambool,217,Warrnambool and South West,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,62.4199,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((142.43668 -38.35544, 142.43658 -38.3..."
1165,217041480,Warrnambool - South,0,No change,21704,Warrnambool,217,Warrnambool and South West,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,120.1957,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((142.45281 -38.39126, 142.45230 -38.3..."
1166,297979799,Migratory - Offshore - Shipping (Vic.),0,No change,29797,Migratory - Offshore - Shipping (Vic.),297,Migratory - Offshore - Shipping (Vic.),29799,Migratory - Offshore - Shipping (Vic.),2,Victoria,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None


In [12]:
sa2_rent_data = sa2_rent_data[['region', 'cost_text']]

## visualization

In [13]:
sa2_rent_data_grouped = sa2_rent_data.groupby('region', as_index=False)['cost_text'].median()

In [14]:
sa2_rent_data_grouped.columns

Index(['region', 'cost_text'], dtype='object')

In [15]:
sa2_rent_data_grouped

,region,cost_text
0,Abbotsford,695.0
1,Airport West,625.0
2,Albert Park,1047.5
3,Albion,500.0
4,Alfredton,500.0
...,...,...
286,Wollert,547.5
287,Wonthaggi,420.0
288,Wyndham Vale,460.0
289,Yarraville,740.0


In [16]:
victoria_sa2 = victoria_sa2.merge(sa2_rent_data_grouped, left_on='SA2_NAME21', right_on='region', how='left').dropna()

In [17]:
victoria_sa2 = victoria_sa2.loc[:, ~victoria_sa2.columns.duplicated()]

In [18]:
victoria_sa2

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry,region,cost_text
0,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5...",Alfredton,500.00
2,201011005,Buninyong,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,51.5855,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.84171 -37.61596, 143.84176 -37.6...",Buninyong,400.00
3,201011006,Delacombe,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,34.1607,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.75050 -37.59119, 143.75044 -37.5...",Delacombe,450.00
19,202011018,Bendigo,0,No change,20201,Bendigo,202,Bendigo,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,16.0796,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((144.24392 -36.74285, 144.24276 -36.7...",Bendigo,420.00
38,203021037,Belmont,0,No change,20302,Geelong,203,Geelong,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,9.3677,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((144.32516 -38.17643, 144.32431 -38.1...",Belmont,560.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,216021414,Yarrawonga,0,No change,21602,Moira,216,Shepparton,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,94.6968,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((146.00051 -36.00877, 146.00128 -36.0...",Yarrawonga,500.00
506,216031593,Kialla,1,New,21603,Shepparton,216,Shepparton,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,77.4435,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((145.37516 -36.42180, 145.37525 -36.4...",Kialla,500.00
510,217011422,Portland,0,No change,21701,Glenelg - Southern Grampians,217,Warrnambool and South West,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,63.9850,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((141.54206 -38.34568, 141.54185 -38.3...",Portland,1955.36
512,217031471,Camperdown,0,No change,21703,Colac - Corangamite,217,Warrnambool and South West,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,98.1179,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.09968 -38.22906, 143.09858 -38.2...",Camperdown,450.00


In [19]:
m = folium.Map(location=[-37.81, 144.96], zoom_start=6, tiles="cartodbpositron")
geojson_data = json.loads(victoria_sa2.to_json())

folium.Choropleth(
    geo_data=geojson_data,
    name='choropleth',
    data=victoria_sa2,
    columns=['SA2_NAME21', 'cost_text'],
    key_on='feature.properties.SA2_NAME21',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Median Rent Cost'
).add_to(m)

top_10_regions = sa2_rent_data_grouped.nlargest(10, 'cost_text')

for idx, row in top_10_regions.iterrows():
    region_name = row['region']
    avg_cost = row['cost_text']
    
    matching_row = victoria_sa2[victoria_sa2['SA2_NAME21'] == region_name]
    if not matching_row.empty:
        centroid = matching_row.geometry.centroid.iloc[0]
        
        folium.Marker(
            location=[centroid.y, centroid.x],
            icon=folium.DivIcon(
                html=f'<div style="font-size: 12px; color: black; padding: 2px;">{region_name}: {avg_cost:.2f}</div>'
            )
        ).add_to(m)

m.save('../../plots/geo(html)/Victoria_Median_Rent_Cost_SA2.html')

/var/folders/kt/yhd0mp_91gn1qrk_l__r080r0000gn/T/ipykernel_27068/3995094748.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = matching_row.geometry.centroid.iloc[0]
/var/folders/kt/yhd0mp_91gn1qrk_l__r080r0000gn/T/ipykernel_27068/3995094748.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = matching_row.geometry.centroid.iloc[0]
/var/folders/kt/yhd0mp_91gn1qrk_l__r080r0000gn/T/ipykernel_27068/3995094748.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = matching_row.geometry.centroid.iloc[0]
/var/folders/kt/yhd0mp_91gn1qrk_l__r080r